<a href="https://colab.research.google.com/github/unique-subedi/gene-expression/blob/main/Unique_gene_expression_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import datetime
import time
import math
import numpy as np
from numpy import linalg as LA
import pandas as pd
import urllib.request

In [2]:
!pip install pyreadr
import pyreadr
urllib.request.urlretrieve("https://raw.githubusercontent.com/unique-subedi/gene-expression/main/data/brain.rda", "brain.rda")
brain = pyreadr.read_r("brain.rda")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 363 kB 5.1 MB/s 


In [3]:
expression = pd.DataFrame(brain["expression"])
genes = pd.DataFrame(brain["genes"])
samples = pd.DataFrame(brain["samples"])

In [4]:
expression.head()
genes.head()
samples.iloc[:20,:]

,patient,sex,region,lab,chip.version
rownames,,,,,
01_a_D_f_2.CEL,patient_01,female,A.C. cortex,Davis,v2
01_a_I_f_2.CEL,patient_01,female,A.C. cortex,Irvine,v2
01_a_M_f_1.CEL,patient_01,female,A.C. cortex,Michigan,v1
01_c_D_f_1.CEL,patient_01,female,cerebellum,Davis,v1
01_c_I_f_2.CEL,patient_01,female,cerebellum,Irvine,v2
01_c_M_f_1.CEL,patient_01,female,cerebellum,Michigan,v1
01_d_I_f_2.CEL,patient_01,female,D.L.P.F. cortex,Irvine,v2
01_d_M_f_1.CEL,patient_01,female,D.L.P.F. cortex,Michigan,v1
02_a_I_m_2.CEL,patient_02,male,A.C. cortex,Irvine,v2


`michigan` dataset contains the measurements, done at University of Michigan lab, of anterior cortex and dorsolateral cortex for each patient. 

`michigan_diff` subtracts measurement of anterior cortex from the dorsolateral cortex for each patient.

# Michigan

In [5]:
michigan = expression[(expression.index.str[5] == 'M') & (expression.index.str[3] != 'c') ]
michigan_diff = michigan.diff().iloc[1::2,:]
michigan_diff.index = "patient" + michigan_diff.index.str[:2]
michigan_diff

,1000_at,1001_at,1002_f_at,1003_s_at,1004_at,1005_at,1006_at,1007_s_at,1008_f_at,1009_at,...,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at,AFFX-YEL002c/WBP1_at,AFFX-YEL018w/_at,AFFX-YEL021w/URA3_at,AFFX-YEL024w/RIP1_at,AFFX-hum_alu_at
patient01,1.683506,1.078551,0.954802,1.156709,0.982425,1.161395,0.890784,1.828306,1.124522,2.052794,...,0.732205,0.717167,0.600567,0.713971,0.576473,0.703452,0.676053,0.591423,0.763982,1.062522
patient02,0.996391,0.544034,0.358586,0.683919,0.587913,0.984146,0.349759,0.993259,1.090844,1.364808,...,0.594352,0.432579,0.375512,0.516236,0.421621,0.533885,0.481034,0.352948,0.443977,0.724038
patient03,0.748157,0.410856,0.375251,0.664853,0.519776,0.806980,0.566739,0.900413,1.118494,1.007720,...,0.504707,0.452460,0.378896,0.451803,0.356578,0.436162,0.509169,0.479238,0.485842,0.763825
patient04,0.965669,0.526189,0.348966,0.575349,0.353175,0.445199,0.361394,1.390621,0.703409,1.171899,...,0.332539,0.311592,0.209239,0.335892,0.255916,0.270513,0.342929,0.264286,0.342026,0.986904
patient05,0.805991,0.436910,0.485836,0.179388,0.270416,0.651277,0.387753,0.223662,0.560202,0.392694,...,0.396176,0.366402,0.532929,0.333441,0.287466,0.428850,0.365462,0.462509,0.414235,-0.063161
patient06,0.523212,0.548558,0.332666,0.131101,0.105857,0.221342,0.260256,0.164966,-0.164174,0.405472,...,0.330896,0.188302,0.309652,0.365562,0.230068,0.495998,0.621407,0.600562,0.542592,-0.431859
patient07,0.744211,0.529751,0.570855,0.402839,0.382700,0.699813,0.474074,0.725456,0.385778,0.179182,...,0.616852,0.600624,0.649939,0.529744,0.603646,0.613554,0.463463,0.596304,0.379135,0.208659
patient08,0.307347,0.097159,0.261037,-0.244442,-0.034621,-0.026341,-0.008480,0.135303,-0.040331,-0.086372,...,0.055827,0.091226,0.225278,0.022298,0.051644,0.089153,-0.030807,0.369845,-0.048141,0.075750
patient09,0.632714,0.512219,0.452847,0.249875,0.268599,0.495261,0.314457,0.360264,0.214071,-0.038456,...,0.502343,0.394397,0.472787,0.441500,0.462422,0.510486,0.483740,0.581277,0.485722,0.255156
patient10,0.817703,0.410601,0.404771,0.237337,0.246678,0.382905,0.382378,0.391850,0.231995,0.225793,...,0.451583,0.405504,0.511481,0.502447,0.503460,0.519477,0.444961,0.559787,0.526603,0.080064


# Davis

In [ ]:
davis = expression[(expression.index.str[5] == 'D') & (expression.index.str[3] != 'c') ]
davis = davis[~davis.index.str[1].isin(['1', '2', '5', '6', '7', '9'])]
davis

,1000_at,1001_at,1002_f_at,1003_s_at,1004_at,1005_at,1006_at,1007_s_at,1008_f_at,1009_at,...,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at,AFFX-YEL002c/WBP1_at,AFFX-YEL018w/_at,AFFX-YEL021w/URA3_at,AFFX-YEL024w/RIP1_at,AFFX-hum_alu_at
03_a_D_f_2.CEL,10.209852,7.485236,7.234562,8.314890,8.052416,8.925883,6.913494,9.978907,9.570872,10.333585,...,7.307212,6.879768,6.621575,7.105408,7.205002,6.935748,6.924961,7.330380,6.860821,14.269806
03_d_D_f_2.CEL,9.811333,7.455546,7.076209,7.884940,7.753675,8.613655,6.917695,9.384268,9.109613,10.459857,...,7.019459,6.744701,6.610451,6.914325,6.875251,6.738741,6.744836,6.819508,6.881588,14.271549
04_a_D_m_2.CEL,10.042199,7.731652,7.622907,8.272572,8.205198,8.747379,7.421068,9.764135,9.448685,10.149796,...,7.509026,7.328737,7.087150,7.440248,7.506122,7.402372,7.298852,7.590137,7.509243,14.102170
04_d_D_m_2.CEL,10.088771,7.965398,7.693108,8.376746,8.342941,8.530199,7.516385,9.766547,9.402945,9.994229,...,7.720471,7.406051,7.251605,7.512881,7.518005,7.398756,7.417299,7.833770,7.523324,14.181253
08_a_D_f_2.CEL,8.583730,6.786933,6.604774,7.247992,7.133094,7.337619,6.598450,8.272535,9.045322,9.737900,...,6.635843,6.551977,6.427348,6.578013,6.683232,6.463394,6.445941,7.020327,6.498995,13.380732
08_d_D_f_2.CEL,10.165526,7.837531,7.497092,8.277114,8.238780,8.870164,7.397192,9.612996,9.407036,10.579853,...,7.717791,7.474512,7.278003,7.677567,7.672219,7.470636,7.384519,7.844122,7.565043,14.185495
10_a_D_f_2.CEL,9.583039,7.516411,7.175049,7.777833,7.839906,8.082034,7.118302,9.364017,9.296023,8.944335,...,7.240170,7.067227,6.921529,7.177594,7.261505,6.939956,7.060879,7.268496,7.093494,14.035990
10_d_D_f_2.CEL,9.143998,6.862450,6.357731,7.134995,7.046711,7.730850,6.183967,9.414389,9.065153,8.863422,...,6.258799,6.027364,5.907490,6.171226,6.130500,6.080228,5.966166,6.135691,6.023620,14.244014


In [ ]:
davis_diff = davis.diff().iloc[1::2,:]
davis_diff.index = "patient" + davis_diff.index.str[:2]
davis_diff

,1000_at,1001_at,1002_f_at,1003_s_at,1004_at,1005_at,1006_at,1007_s_at,1008_f_at,1009_at,...,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at,AFFX-YEL002c/WBP1_at,AFFX-YEL018w/_at,AFFX-YEL021w/URA3_at,AFFX-YEL024w/RIP1_at,AFFX-hum_alu_at
patient03,-0.398519,-0.029690,-0.158353,-0.429949,-0.298741,-0.312228,0.004201,-0.594639,-0.461260,0.126272,...,-0.287753,-0.135067,-0.011124,-0.191083,-0.329751,-0.197007,-0.180125,-0.510872,0.020767,0.001743
patient04,0.046573,0.233745,0.070201,0.104174,0.137743,-0.217180,0.095317,0.002412,-0.045740,-0.155568,...,0.211445,0.077314,0.164455,0.072634,0.011883,-0.003616,0.118447,0.243633,0.014081,0.079084
patient08,1.581795,1.050599,0.892318,1.029122,1.105686,1.532545,0.798743,1.340461,0.361714,0.841953,...,1.081948,0.922535,0.850656,1.099555,0.988987,1.007243,0.938578,0.823795,1.066048,0.804763
patient10,-0.439041,-0.653961,-0.817318,-0.642838,-0.793195,-0.351184,-0.934335,0.050372,-0.230870,-0.080913,...,-0.981371,-1.039864,-1.014039,-1.006368,-1.131005,-0.859728,-1.094714,-1.132805,-1.069875,0.208024


# Irvine

In [9]:
irvine = expression[(expression.index.str[5] == 'I') & (expression.index.str[3] != 'c') ]
#irvine = irvine[~irvine.index.str[1].isin(['1', '2', '5', '6', '7', '9'])]
irvine_diff = irvine.diff().iloc[1::2,:]
irvine_diff.index = "patient" + irvine_diff.index.str[:2]
irvine_diff

,1000_at,1001_at,1002_f_at,1003_s_at,1004_at,1005_at,1006_at,1007_s_at,1008_f_at,1009_at,...,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at,AFFX-YEL002c/WBP1_at,AFFX-YEL018w/_at,AFFX-YEL021w/URA3_at,AFFX-YEL024w/RIP1_at,AFFX-hum_alu_at
patient01,0.462602,0.196471,0.037546,0.158286,0.209469,0.304693,0.047545,0.517627,-0.032488,0.655393,...,0.134520,0.089560,0.129932,0.141098,0.129605,0.140019,0.096710,0.012792,0.207097,-0.000063
patient02,0.937158,0.700623,0.711439,0.769030,0.624114,0.795289,0.688129,0.841179,0.340079,1.200148,...,1.407043,1.354821,1.177115,1.286127,1.083917,0.669673,0.653216,0.549765,0.684012,-0.000594
patient03,-0.141804,-0.013418,-0.105409,-0.222431,-0.132847,-0.061719,-0.012419,-0.491726,0.221750,-0.233455,...,0.001451,-0.091015,0.067013,-0.007767,-0.030427,0.010075,0.035978,0.078626,0.044217,0.000094
patient04,-0.809163,-0.529402,-0.420761,-0.514874,-0.447364,-0.788638,-0.378308,-0.860960,-0.437358,-0.869543,...,-0.253332,-0.328701,-0.117450,-0.355506,-0.229959,-0.192604,-0.234400,-0.296673,-0.186457,0.000219
patient05,-0.003735,0.027986,0.168002,0.193760,0.131893,0.301568,0.130296,-0.341332,0.397826,0.212857,...,0.006575,0.040954,0.071474,-0.052911,0.030074,0.051787,0.058027,-0.046889,0.064747,0.000094
patient06,0.421542,0.941550,0.709334,0.696667,0.867351,0.432269,0.724171,-0.117563,0.449276,0.719218,...,0.821821,0.802139,0.792695,0.805005,0.857420,0.880585,0.783537,0.577158,0.659995,0.000063
patient07,-0.022561,-0.036318,-0.147432,0.006680,-0.046469,0.298594,-0.158007,0.040739,0.111852,-0.039771,...,0.032205,-0.015521,-0.213513,-0.017854,-0.088799,-0.085034,-0.072329,-0.013949,0.066306,-0.000344
patient08,0.076724,-0.075931,0.109135,0.137162,0.013977,-0.123317,-0.006490,-0.038578,0.077426,-0.060147,...,0.058803,0.127142,0.089811,0.112357,0.043009,0.149965,0.151649,0.072133,0.161168,-0.000250
patient09,-0.155818,-0.083225,-0.071142,-0.140460,0.050254,0.056232,-0.054453,-0.360464,-0.116619,-0.155538,...,-0.174063,-0.119373,-0.008352,-0.150368,-0.095377,-0.061853,-0.066803,-0.074266,-0.125916,0.000531
patient10,-0.273780,-0.242814,-0.044002,-0.139106,-0.065981,-0.323471,-0.139116,-0.369869,-0.138864,-0.034904,...,-0.107692,-0.172641,-0.051986,-0.063974,0.041484,-0.194847,-0.158551,-0.151667,-0.006174,0.000063


In [ ]:
(michigan_diff.loc['patient08']< 0).sum()

4176

In [ ]:
michigan_diff.loc[:, '33942_s_at']

patient01    3.089106
patient02    1.849506
patient03    1.621996
patient04    1.514725
patient05    1.296588
patient06    1.890518
patient07    1.154332
patient08    0.658243
patient09    1.048392
patient10    1.169494
Name: 33942_s_at, dtype: float64

Out of 126000 entries in this dataset, only 6764 (~5%) of the entries are negative. That means, the measurement for dorsolateral region is generally larger than that of anterior region for every patient.

In [ ]:
126000 - (michigan_diff > 0).sum().sum()

6764

Take absolute values of each difference and compute the average over all patients for each probe sets and obtain top 20 probe sets with highest magnitude.

In [ ]:
top_probes_michigan = michigan_diff.abs().mean().sort_values(ascending = False).iloc[:100]
top_probes_michigan

38308_g_at    1.700100
33925_at      1.581787
33942_s_at    1.529290
36608_at      1.494925
35742_at      1.435630
                ...   
41498_at      1.140076
38708_at      1.137271
32315_at      1.136706
40512_at      1.134779
35174_i_at    1.133433
Length: 100, dtype: float64

Map top probe sets to top genes using `genes` dataset. 

In [ ]:
top_genes_michigan = genes.loc[top_probes_michigan.keys()].loc[:, 'sym'].values
top_genes_michigan

['NCDN', 'NRGN', 'STXBP1', 'MDH1', 'C16orf45', ..., 'HNRNPK', 'NPTN', 'FAM127A', 'OLFM1', 'VAMP2']
Length: 20
Categories (8783, object): ['AADAC', 'AAK1', 'AAMP', 'AANAT', ..., 'ZYX', 'ZZEF1', 'ZZZ3', 'psiTPTE22']

In [ ]:
def get_top_genes(lab):
  if(lab == 'M' or lab == 'I'):
    df = expression[(expression.index.str[5] == lab) & (expression.index.str[3] != 'c') & ~(expression.index.str[1].isin(['1', '2', '5', '6', '7', '9'])) ]
    df_diff = df.diff().iloc[1::2,:]
  
  else:
    df = expression[(expression.index.str[5] == lab) & (expression.index.str[3] != 'c') & ~(expression.index.str[1].isin(['1', '2', '5', '6', '7', '9'])) ]
    df_diff = df.diff().iloc[1::2,:]
  top_probes = df_diff.abs().mean().sort_values(ascending = False).iloc[:20]
  top_genes = genes.loc[top_probes.keys()].loc[:, 'sym'].values
  return np.array(top_genes)


In [ ]:
top_genes_michigan = get_top_genes("M")
top_genes_irvine = get_top_genes("I")
top_genes_davis = get_top_genes("D")
print(top_genes_irvine)
print(top_genes_michigan)
print(top_genes_davis)

['APOE' 'NEFH' 'VIM' 'NEFM' 'CALM3' 'SLC17A7' 'NTRK2' 'MT1G' 'ATP1B2'
 'PTGDS' 'C1orf61' 'NTRK2' 'GRINA' 'PPP2R4' 'FAM107A' 'GFAP' 'SCN1B'
 'SLC17A7' 'NEFL' 'SELENBP1']
['NCDN' 'CBX6' 'NRGN' 'TSPYL4' 'HNRNPK' 'EEF1A2' 'GAS7' 'CLASP2' 'MDH1'
 'CALM2' 'TUBB4' 'NPTN' 'JUND' 'C16orf45' 'AP2M1' 'WBP2' 'OLFM1' 'KIF5C'
 nan 'CALM3']
['ACTB' 'GAPDH' 'ACTB' 'MT3' 'CABP1' 'RHOB' 'TMBIM6' 'UBE2M' 'WNT6' 'CNP'
 'PLD3' 'CALM1' 'CCDC85B' 'NCDN' 'LGALS1' 'AES' 'EEF1A2' 'B2M' 'SST'
 'THRA']


## Hypothesis testing 
Wilcoxon signed-rank test for each gene(Remove controls).  
Tests whether the median of the difference is 0.  
A non-parametric alternative test to the paired t-test. 

In [11]:
from scipy.stats import wilcoxon
from statsmodels.stats.multitest import multipletests
# Remove controls(Probesets starting with 'AFFX')
michigan_diff = michigan_diff.filter(regex='^(?!AFFX).*$', axis=1)
irvine_diff = irvine_diff.filter(regex='^(?!AFFX).*$', axis=1)


p_val_M = michigan_diff.apply(wilcoxon).iloc[1,:]
print('Michigan')
print('rejected H0(bonferroni):{}'.format(sum(multipletests(p_val_M, method='bonferroni')[0])))
print('rejected H0(BH):{} \n'.format(sum(multipletests(p_val_M, method='fdr_bh')[0])))


p_val_I = irvine_diff.apply(wilcoxon).iloc[1,:]
print('Irvine')
print('rejected H0(bonferroni):{}'.format(sum(multipletests(p_val_I, method='bonferroni')[0])))
print('rejected H0(BH):{} \n'.format(sum(multipletests(p_val_I, method='fdr_bh')[0])))

Michigan
rejected H0(bonferroni):0
rejected H0(BH):11874 

Irvine
rejected H0(bonferroni):0
rejected H0(BH):0 



In [14]:
print(p_val_M.sort_values(ascending = True).iloc[:100])

1000_at       0.001953
35802_at      0.001953
35801_at      0.001953
39994_at      0.001953
357_at        0.001953
                ...   
40034_r_at    0.001953
35700_at      0.001953
35731_at      0.001953
35732_at      0.001953
35734_at      0.001953
Name: 1, Length: 100, dtype: float64


In [15]:
print(p_val_I.sort_values(ascending = True).iloc[:100])

39031_at      0.013672
35457_at      0.019531
34281_at      0.037109
40202_at      0.048828
31997_at      0.048828
                ...   
39790_at      0.130859
36759_at      0.130859
2007_g_at     0.130859
34938_i_at    0.130859
1606_at       0.130859
Name: 1, Length: 100, dtype: float64
